In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 21.8 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824025 sha256=c9bbeb5820fd96d685e4463b66fd1caaae845d0521830de95232e6e020d70658
  Stored in directory: /root/.cache/pip/wheels/b1/59/a0/a1a0624b5e865fd389919c1a10f53aec9b12195d6747710baf
Successfully built pyspark


In [2]:
import pyspark

In [3]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('Practise').getOrCreate()

In [22]:
##read the dataset
df_pyspark=spark.read.format("csv").options(delimiter=";", header=True ).load("/content/drive/MyDrive/Pypask_Course/test4Columns.csv")

In [5]:
df_pyspark.show()

+-----+----+----------+------+
| name| age|experience|salary|
+-----+----+----------+------+
|name1|  30|         1|  3000|
|name2|  30|         2|   201|
|name3|  31|         3|   202|
|name4|  31|         3|   203|
|name5|  31|         3|   203|
|name6|  31|         3|   203|
|name7|null|      null|   203|
| null|  31|         3|   203|
| null|  31|      null|  null|
+-----+----+----------+------+



####Eliminar columna y valores nulos de las columnas

In [7]:
df_pyspark.drop('name').show()

+----+----------+------+
| age|experience|salary|
+----+----------+------+
|  30|         1|  3000|
|  30|         2|   201|
|  31|         3|   202|
|  31|         3|   203|
|  31|         3|   203|
|  31|         3|   203|
|null|      null|   203|
|  31|         3|   203|
|  31|      null|  null|
+----+----------+------+



In [8]:
df_pyspark.na.drop().show()

+-----+---+----------+------+
| name|age|experience|salary|
+-----+---+----------+------+
|name1| 30|         1|  3000|
|name2| 30|         2|   201|
|name3| 31|         3|   202|
|name4| 31|         3|   203|
|name5| 31|         3|   203|
|name6| 31|         3|   203|
+-----+---+----------+------+



na.drop(how="any") es utilizado en PySpark para eliminar todas las filas que contienen valores nulos o faltantes en un DataFrame.

In [9]:
#df_pyspark.na.drop(how="any", thresh=2) elimina todas las filas que contienen al menos un valor nulo (es decir, "NaN")
 #en el DataFrame df_pyspark, pero solo después de haber verificado que al menos dos valores no nulos se encuentran
  #en cada fila.
df_pyspark.na.drop(how="any" , thresh=2).show()

+-----+----+----------+------+
| name| age|experience|salary|
+-----+----+----------+------+
|name1|  30|         1|  3000|
|name2|  30|         2|   201|
|name3|  31|         3|   202|
|name4|  31|         3|   203|
|name5|  31|         3|   203|
|name6|  31|         3|   203|
|name7|null|      null|   203|
| null|  31|         3|   203|
+-----+----+----------+------+



In [10]:
#df_pyspark.na.drop(how="any" , subset=['experience']).show() elimina datos nulos de una columan especifica
df_pyspark.na.drop(how="any" , subset=['experience']).show()

+-----+---+----------+------+
| name|age|experience|salary|
+-----+---+----------+------+
|name1| 30|         1|  3000|
|name2| 30|         2|   201|
|name3| 31|         3|   202|
|name4| 31|         3|   203|
|name5| 31|         3|   203|
|name6| 31|         3|   203|
| null| 31|         3|   203|
+-----+---+----------+------+



In [12]:
#llenando valores nulos
#df_pyspark.na.fill('mising values :)').show() #Para todas las columnas
#df_pyspark.na.fill('mising values :)',['experience','name']).show() 
df_pyspark.na.fill('mising values :)','experience').show() #para colum especifica

+-----+----+----------------+------+
| name| age|      experience|salary|
+-----+----+----------------+------+
|name1|  30|               1|  3000|
|name2|  30|               2|   201|
|name3|  31|               3|   202|
|name4|  31|               3|   203|
|name5|  31|               3|   203|
|name6|  31|               3|   203|
|name7|null|mising values :)|   203|
| null|  31|               3|   203|
| null|  31|mising values :)|  null|
+-----+----+----------------+------+



In [27]:
from pyspark.sql.functions import col
df_pyspark = df_pyspark.withColumn("age", col("age").cast("integer"))
df_pyspark = df_pyspark.withColumn("experience", col("experience").cast("integer"))
df_pyspark = df_pyspark.withColumn("salary", col("salary").cast("integer"))

In [28]:
from pyspark.ml.feature import Imputer

imputer = Imputer(
    inputCols=['age', 'experience', 'salary'], 
    outputCols=["{}_imputed".format(c) for c in ['age', 'experience', 'salary']]
    ).setStrategy("median")

In [29]:
# Add imputation cols to df
imputer.fit(df_pyspark).transform(df_pyspark).show()

+-----+----+----------+------+-----------+------------------+--------------+
| name| age|experience|salary|age_imputed|experience_imputed|salary_imputed|
+-----+----+----------+------+-----------+------------------+--------------+
|name1|  30|         1|  3000|         30|                 1|          3000|
|name2|  30|         2|   201|         30|                 2|           201|
|name3|  31|         3|   202|         31|                 3|           202|
|name4|  31|         3|   203|         31|                 3|           203|
|name5|  31|         3|   203|         31|                 3|           203|
|name6|  31|         3|   203|         31|                 3|           203|
|name7|null|      null|   203|         31|                 3|           203|
| null|  31|         3|   203|         31|                 3|           203|
| null|  31|      null|  null|         31|                 3|           203|
+-----+----+----------+------+-----------+------------------+--------------+